### 0. Load the input data

In [ ]:
conda activate bioinfo
pip install pyscenic
conda install -y numpy

In [1]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

In [2]:
from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

/anaconda3/envs/scenic_protocol/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
DATA_FOLDER="/Users/apple/Desktop/work/data/SCENIC/OB"
RESOURCES_FOLDER="/Users/apple/Desktop/work/data/SCENIC/数据"
DATABASE_FOLDER = "/Users/apple/Desktop/work/data/SCENIC/cisTarget_databases/"
SCHEDULER="123.122.8.24:8786"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_mgi_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "exprMat_filtered_OB.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")

In [ ]:
#服务器
DATA_FOLDER="/home/lmlu/scRNAseq/SCENIC/OB"
RESOURCES_FOLDER="/home/lmlu/scRNAseq/SCENIC/data"
DATABASE_FOLDER = "/home/lmlu/scRNAseq/SCENIC/cisTarget_databases/"
SCHEDULER="123.122.8.24:8786"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_mgi_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "exprMat_filtered_OB.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")

In [ ]:
####### OB neuron
DATA_FOLDER="/home/lmlu/scRNAseq/SCENIC/ob_neuron_male"
RESOURCES_FOLDER="/home/lmlu/scRNAseq/SCENIC/data"
DATABASE_FOLDER = "/home/lmlu/scRNAseq/SCENIC/cisTarget_databases/"
SCHEDULER="123.122.8.24:8786"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_mgi_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "exprMat_OB_neuron_male.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons_neuron_male.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs_neuron_male.csv")

In [5]:
# 读入表达矩阵，表达矩阵的格式：横坐标是基因，纵坐标是细胞
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep='\t', header=0, index_col=0).T
ex_matrix.shape

(4851, 15836)

In [6]:
# 导入转录因子
tf_names = load_tf_names(MM_TFS_FNAME)

# 导入数据库
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.basename(fname).split(".")[0]   
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="mm9-500bp-upstream-7species"),
 FeatherRankingDatabase(name="mm9-tss-centered-10kb-7species")]

In [7]:
ex_matrix.head()

,Sox17,Mrpl15,Lypla1,Tcea1,Rgs20,Atp6v1h,Oprk1,Npbwr1,Rb1cc1,4732440D04Rik,...,mt-Nd5,mt-Nd6,mt-Cytb,Vamp7,Spry3,Tmlhe,AC132444.6,AC125149.3,PISD,DHRSX
AAACCTGAGTTGTAGA_1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.300059,...,1.300059,0.0,4.133546,1.300059,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGTCGGCGCAT_1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,2.640343,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGTCTACTCAT_1,0.0,0.826952,0.0,1.273315,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.826952,0.0,3.645550,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGTCTGACCTC_1,0.0,0.000000,0.0,1.514823,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,2.091522,0.000000,0.0,0.0,0.0,0.0,1.514823,0.0
AAACGGGAGAGAGCTC_1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.541010,0.0,4.085345,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [8]:
tf_names[:5]

['Hoxa9', 'Zfp128', 'Zfp853', 'Nr1h2', 'Nr1h3']

### 1. Inference of co-expression modules

In [9]:
# 两条命令解决
adjacencies = grnboost2(ex_matrix, tf_names=tf_names, verbose=True) #耗时

preparing dask client
parsing input
creating dask graph


/anaconda3/envs/scenic_protocol/lib/python3.6/site-packages/arboreto/algo.py:214: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  expression_matrix = expression_data.as_matrix()


4 partitions
computing dask graph
shutting down client and local cluster
finished


In [10]:
adjacencies.head()

,TF,target,importance
1152,Mylk,Tagln,205.518957
1152,Mylk,Pln,162.223438
1152,Mylk,Myh11,159.591700
308,Jun,Fos,140.834201
1066,Fos,Jun,139.244663


In [ ]:
adjacencies.to_csv('OB_network.tsv', sep='\t', header=False, index=False)

In [11]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix))


2020-07-22 00:33:26,000 - pyscenic.utils - INFO - Calculating Pearson correlations.

2020-07-22 00:33:28,431 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2020-07-22 00:35:25,207 - pyscenic.utils - INFO - Creating modules.


In [12]:
modules[:10]

[Regulon(name='Regulon for 1810024B03Rik', gene2weight=<frozendict {'Zp3': 7.189295875846311, 'Trim36': 5.1916712039154325, 'Nudcd2': 5.0701201708671055, 'Shank2': 4.998584856320017, 'Lrrc36': 4.644074837725707, 'Ccdc151': 4.267150845199709, 'Cmbl': 4.25369651405648, 'Smim5': 4.198711597810231, 'Dthd1': 3.8714232198485314, 'Dnali1': 3.4709228505434755, 'Lrrc71': 3.225507935407718, 'Zmynd19': 2.9759039118056734, 'S100a3': 2.9625572998069036, '1700016K19Rik': 2.844811955950031, 'I830077J02Rik': 2.7884249378108485, 'Cfap46': 2.782250601994094, 'Lrrc51': 2.738360567223428, 'Malt1': 2.6676074161706143, 'Sowaha': 2.571687954237319, 'Dok1': 2.5048688700647457, 'Zbtb46': 2.489267893146697, 'Tmem68': 2.4853365081531194, 'Pon3': 2.4785920423201118, 'Erich2': 2.3946657611677304, 'Odf3b': 2.3893230769851095, 'Drg1': 2.306056802177854, 'Fam216b': 2.302001875213974, 'Tnfsf13os': 2.2465659755103258, 'Gm42669': 2.180040091919426, '1110017D15Rik': 2.170285551882759, 'Ms4a6b': 2.151902789899293, 'Col26a

### 2. Prune modules for targets with cis regulatory footprints (aka RcisTarget)

In [ ]:
# Calculate a list of enriched motifs and the corresponding target genes for all modules.
with ProgressBar():
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME)

# Create regulons from this table of enriched motifs.
regulons = df2regulons(df)

# Save the enriched motifs and the discovered regulons to disk.
df.to_csv(MOTIFS_FNAME)
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons, f)

[                                        ] | 0% Completed | 11.6s


2020-07-22 00:38:58,762 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for 9030624G23Rik could be mapped to mm9-500bp-upstream-7species. Skipping this module.


[                                        ] | 0% Completed | 18.0s


2020-07-22 00:39:05,220 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for Adnp could be mapped to mm9-500bp-upstream-7species. Skipping this module.


[                                        ] | 0% Completed |  1min 50.6s

### 3. Cellular regulon enrichment matrix (aka AUCell)

In [ ]:
auc_mtx = aucell(ex_matrix, regulons, num_workers=4)
# 这一步出图
# sns.clustermap(auc_mtx, figsize=(8,8))

In [ ]:
auc_mtx.to_csv(AUC_FNAME)

## pySCENIC结果导入R

In [ ]:
auc_mtx = pd.read_csv("/home/lmlu/scRNAseq/SCENIC/OB/auc.tsv", index_col=0)

In [ ]:
with open(REGULONS_FNAME,"rb") as f:
    regulons=pickle.load(f)

In [ ]:
#保存为.loom格式
from pyscenic.export import export2loom
export2loom(ex_mtx = ex_matrix, auc_mtx = auc_mtx, regulons = regulons, out_fname = "/home/lmlu/scRNAseq/SCENIC/OB/OB.loom")
# 这里会有提示（见图1），按照提示改一下

export2loom(ex_mtx = ex_matrix, auc_mtx = auc_mtx, regulons = [r.rename(r.name.replace('(+)',' ('+str(len(r))+'g)')) for r in regulons], out_fname = "/home/lmlu/scRNAseq/SCENIC/OB/OB.loom")
# 这一句话运行完毕后，会在指定目录下生成xxx.loom文件，这就是导入R所需要的文件